In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "OpenLLM-Ro/RoLlama3-8b-Instruct-DPO"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype="auto"
)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/539 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/795 [00:00<?, ?B/s]

...

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/206 [00:00<?, ?B/s]

In [ ]:
import random
import json

# ==========================
#     TEMPLATE & CONFIG
# ==========================

SYSTEM_MSG = """Ești un asistent care redactează cereri juridice clare, politicoase și bine structurate în limba română, respectând toate cerințele formale impuse de lege."""

TEMPLATE = """
Redactează o cerere juridică completă în limba română, pe baza următoarelor elemente:
- TipCerere: {tip}
- Parti implicate: {parti}
- Context/Scop: {scop}
- Conflict juridic: {conflict}
- Argumente și probe: {argumente}
- Termen dorit: {termen}
- Ton: {ton}
- Lungime aproximativă: {lungime} cuvinte

Structura cererii trebuie să includă:
- Titlu
- Preambul
- Corpul documentului (cu articole/paragrafe numerotate)
- Semnături
- Dată
- Ștampile (dacă este cazul)

Asigură respectarea cerințelor legale și a formalităților prevăzute de lege pentru cererile juridice.
"""

types = ["chemare în judecată", "apel", "recurs"]
parti = ["persoană fizică", "persoană juridică"]
scopuri = ["reglementare", "dovadă", "soluționare a unui conflict", "constatare a unui fapt"]
conflict = ["există conflict juridic", "nu este cazul unui conflict juridic"]
argumente = [
    "prezint probele anexate",
    "invoc prevederile legale aplicabile",
    "anexez înscrisuri justificative",
    "menționez circumstanțele faptice relevante"
]
termeni = ["de urgență", "în termen legal", "până la primul termen de judecată"]
tonuri = ["formal", "strict formal"]
lungimi = [200, 220, 210]


# =====================================================
#   GENERARE PROMPT + META-DATE CONSISTENTE
# =====================================================

def generate_prompt_and_meta():

    meta = {
        "TipCerere": random.choice(types),
        "PartiImplicate": random.choice(parti),
        "Context": random.choice(scopuri),
        "Conflict": random.choice(conflict),
        "Argumente": random.choice(argumente),
        "Termen": random.choice(termeni),
        "Ton": random.choice(tonuri),
        "Lungime": random.choice(lungimi)
    }

    prompt = TEMPLATE.format(
        tip=meta["TipCerere"],
        parti=meta["PartiImplicate"],
        scop=meta["Context"],
        conflict=meta["Conflict"],
        argumente=meta["Argumente"],
        termen=meta["Termen"],
        ton=meta["Ton"],
        lungime=meta["Lungime"]
    )

    return prompt, meta


# =====================================================
#   MODEL + CURĂȚARE TEXT + SALVARE ÎN JSON
# =====================================================

def call_model(system, prompt, meta, temperature=0.4, max_new_tokens=600):

    full_prompt = f"System: {system}\nUser: {prompt}\nAssistant:"
    inputs = tokenizer(full_prompt, return_tensors="pt").to(model.device)

    outputs = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        temperature=temperature,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id
    )

    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # extrage DOAR textul generat
    generated_only = decoded[len(full_prompt):].strip()

    # curăță \n\n brute → newline reale
    generated_only = generated_only.replace("\\n", "\n")

    # pregătește datele pentru JSON
    data = {
        "TipCerere": meta["TipCerere"],
        "PartiImplicate": meta["PartiImplicate"],
        "Context": meta["Context"],
        "Ton": meta["Ton"],
        "Raspuns": generated_only
    }

    # încarcă istoricul (sau liste gole)
    try:
        with open("istoric_cereri.json", "r", encoding="utf-8") as f:
            existing = json.load(f)
            if not isinstance(existing, list):
                existing = []
    except FileNotFoundError:
        existing = []

    # adaugă la finalul JSON-ului
    existing.append(data)

    # scrie înapoi lista
    with open("istoric_cereri.json", "w", encoding="utf-8") as f:
        json.dump(existing, f, ensure_ascii=False, indent=4)

    return generated_only


# ==========================
#        EXECUȚIE
# ==========================

prompt, meta_data = generate_prompt_and_meta()
#raspuns = call_model(SYSTEM_MSG, prompt, meta_data)

print("=== PROMPT ===\n")
print(prompt)
#print("\n=== RĂSPUNS ===\n")
#print(raspuns)


=== PROMPT ===


Redactează o cerere juridică completă în limba română, pe baza următoarelor elemente:
- TipCerere: recurs
- Parti implicate: persoană fizică
- Context/Scop: constatare a unui fapt
- Conflict juridic: există conflict juridic
- Argumente și probe: anexez înscrisuri justificative
- Termen dorit: în termen legal
- Ton: strict formal
- Lungime aproximativă: 220 cuvinte

Structura cererii trebuie să includă:
- Titlu
- Preambul
- Corpul documentului (cu articole/paragrafe numerotate)
- Semnături
- Dată
- Ștampile (dacă este cazul)

Asigură respectarea cerințelor legale și a formalităților prevăzute de lege pentru cererile juridice.



In [ ]:
for i in range(16):
  prompt, meta_data = generate_prompt_and_meta()
  print("=== PROMPT ===\n")
  print(prompt)
  raspuns = call_model(SYSTEM_MSG, prompt, meta_data)
  print("\n=== RĂSPUNS ===\n")
  print(raspuns)

=== PROMPT ===


Redactează o cerere juridică completă în limba română, pe baza următoarelor elemente:
- TipCerere: recurs
- Parti implicate: persoană juridică
- Context/Scop: dovadă
- Conflict juridic: nu este cazul unui conflict juridic
- Argumente și probe: anexez înscrisuri justificative
- Termen dorit: până la primul termen de judecată
- Ton: formal
- Lungime aproximativă: 200 cuvinte

Structura cererii trebuie să includă:
- Titlu
- Preambul
- Corpul documentului (cu articole/paragrafe numerotate)
- Semnături
- Dată
- Ștampile (dacă este cazul)

Asigură respectarea cerințelor legale și a formalităților prevăzute de lege pentru cererile juridice.


=== RĂSPUNS ===

**CERERE PENTRU RECURS**

**Preambul**

În fața Înaltei Curți de Justiție, [Numele instanței], [Orașul], [Data],

[Partea], reprezentată de [Avocatul], depune această cerere pentru recurs împotriva hotărârii pronunțate de [Numele instanței inferioare], [Orașul], [Data].

**Corpul documentului**

**Articolul 1: Identifica

In [ ]:
for i in range(9):
  prompt, meta_data = generate_prompt_and_meta()
  print("=== PROMPT ===\n")
  print(prompt)
  raspuns = call_model(SYSTEM_MSG, prompt, meta_data)
  print("\n=== RĂSPUNS ===\n")
  print(raspuns)
  from google.colab import files
  files.download('/content/istoric_cereri.json')

=== PROMPT ===


Redactează o cerere juridică completă în limba română, pe baza următoarelor elemente:
- TipCerere: chemare în judecată
- Parti implicate: persoană fizică
- Context/Scop: soluționare a unui conflict
- Conflict juridic: există conflict juridic
- Argumente și probe: invoc prevederile legale aplicabile
- Termen dorit: de urgență
- Ton: formal
- Lungime aproximativă: 200 cuvinte

Structura cererii trebuie să includă:
- Titlu
- Preambul
- Corpul documentului (cu articole/paragrafe numerotate)
- Semnături
- Dată
- Ștampile (dacă este cazul)

Asigură respectarea cerințelor legale și a formalităților prevăzute de lege pentru cererile juridice.


=== RĂSPUNS ===

**TITLU:**
Cerere de soluționare a unui conflict juridic și măsuri urgente

**PREAMBUHL:**
În fața Excelenței Sale, [Numele Judecătorului], avem plăcerea de a depune această cerere de chemare în judecată, în numele [Numele Părții], o persoană fizică, împotriva [Numele Părții Adverse], în legătură cu un conflict juridic 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

=== PROMPT ===


Redactează o cerere juridică completă în limba română, pe baza următoarelor elemente:
- TipCerere: chemare în judecată
- Parti implicate: persoană fizică
- Context/Scop: constatare a unui fapt
- Conflict juridic: există conflict juridic
- Argumente și probe: menționez circumstanțele faptice relevante
- Termen dorit: de urgență
- Ton: formal
- Lungime aproximativă: 210 cuvinte

Structura cererii trebuie să includă:
- Titlu
- Preambul
- Corpul documentului (cu articole/paragrafe numerotate)
- Semnături
- Dată
- Ștampile (dacă este cazul)

Asigură respectarea cerințelor legale și a formalităților prevăzute de lege pentru cererile juridice.


=== RĂSPUNS ===

**CERERE DE INSTITUIRE A PROCESULUI**

**PREAMBUL**

Prin prezenta, [Numele Părții], reprezentată de avocatul său, [Numele Avocatului], depune o cerere de instituire a procesului împotriva [Numele Părții Adverse], pentru a stabili existența unui fapt care a cauzat un conflict juridic.

**CORPUL DOCUMENTULUI (ARTICOLE/

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

=== PROMPT ===


Redactează o cerere juridică completă în limba română, pe baza următoarelor elemente:
- TipCerere: chemare în judecată
- Parti implicate: persoană fizică
- Context/Scop: reglementare
- Conflict juridic: există conflict juridic
- Argumente și probe: prezint probele anexate
- Termen dorit: de urgență
- Ton: strict formal
- Lungime aproximativă: 200 cuvinte

Structura cererii trebuie să includă:
- Titlu
- Preambul
- Corpul documentului (cu articole/paragrafe numerotate)
- Semnături
- Dată
- Ștampile (dacă este cazul)

Asigură respectarea cerințelor legale și a formalităților prevăzute de lege pentru cererile juridice.


=== RĂSPUNS ===

**CERERE DE JUDICIARE**

**PREAMBUL**

În fața onorabilei instanțe, [Numele instanței],

[Numărul de identificare al persoanei fizice], reprezentată de avocatul său [Numele avocatului], depune această cerere de judecată împotriva [Numele părții adverse], în scopul reglementării unui conflict juridic existent între părți.

**ARTICOLUL I: CO

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>